## Datos con ruido 

In [8]:
import os, math, pickle
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from scipy import integrate

# ==================== CONFIGURACIÓN DE PATHS ====================
BASE_DIR = "/home/david/Schr-dingerPINNsUQValidation/PrimeraFase/ValidacionPozoInfinito/FiltroKalman/ResultadosK"
os.makedirs(BASE_DIR, exist_ok=True)

print(f"📁 Directorio base: {BASE_DIR}")
print("✅ Estructura de carpetas creada")

tf.keras.utils.set_random_seed(0)

# ==================== FUNCIÓN DE INTEGRACIÓN COMPATIBLE ====================
def integrate_simps(y, x):
    """
    Función de integración compatible con diferentes versiones de SciPy
    """
    try:
        # Intentar con simps directamente (versiones antiguas)
        from scipy.integrate import simps
        return simps(y, x)
    except ImportError:
        try:
            # Versiones nuevas: simps está en _quadrature
            from scipy.integrate._quadrature import simps
            return simps(y, x)
        except ImportError:
            # Fallback: usar trapezoidal rule de numpy
            return np.trapz(y, x)

# ==================== FILTRO DE KALMAN ====================
class ParameterKalmanFilter:
    """Filtro de Kalman para refinar parámetros durante el entrenamiento"""
    def __init__(self, initial_value, process_noise=0.1, measurement_noise=0.01):
        self.value = initial_value
        self.P = 1.0  # Covarianza del error
        self.Q = process_noise
        self.R = measurement_noise
    
    def update(self, measurement):
        # Predicción
        value_pred = self.value
        P_pred = self.P + self.Q
        
        # Actualización (corrección)
        K = P_pred / (P_pred + self.R)  # Ganancia de Kalman
        self.value = value_pred + K * (measurement - value_pred)
        self.P = (1 - K) * P_pred
        
        return self.value

# ==================== ESTRUCTURA DE LA RED ====================
class BoundaryEnforcerLayer(tf.keras.layers.Layer):
    """
    Capa personalizada para aplicar condiciones de frontera: x * (1 - x)
    """
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        
    def call(self, inputs):
        x = inputs
        # ψ(x) = x * (1 - x) para asegurar ψ(0)=0 y ψ(1)=0
        return x * (1.0 - x)

class TrigNodalFactorLayer(tf.keras.layers.Layer):
    """
    Capa personalizada para calcular el factor nodal trigonométrico
    """
    def __init__(self, n, **kwargs):
        super().__init__(**kwargs)
        self.n = n
        self.pi = tf.constant(math.pi, dtype=tf.float32)
        
    def call(self, inputs):
        x = inputs
        n_float = tf.cast(self.n, dtype=x.dtype)
        
        # Calcular sin(πx) y sin(nπx)
        s1 = tf.sin(self.pi * x)
        sn = tf.sin(n_float * self.pi * x)
        
        # Para evitar división por cero
        epsilon = tf.constant(1e-12, dtype=x.dtype)
        s1_safe = s1 + epsilon
        
        # Calcular el ratio sin(nπx)/sin(πx)
        ratio = sn / s1_safe
        
        # Donde sin(πx) es cercano a cero, usar n (límite de la función)
        mask = tf.abs(s1) < 1e-6
        result = tf.where(mask, n_float, ratio)
        
        return result

def make_net(n=1, hidden=64, use_sine=True):
    """
    Construye la red neuronal
    """
    # Capa de entrada
    x_in = tf.keras.Input(shape=(1,), name='input_layer')
    
    # Aplicar condiciones de frontera: ψ_boundary = x * (1 - x)
    boundary_enforcer = BoundaryEnforcerLayer(name='boundary_enforcer')(x_in)
    
    # Factor nodal trigonométrico
    nodal_factor = TrigNodalFactorLayer(n=n, name='nodal_factor')(x_in)
    
    # Red neuronal principal
    if use_sine:
        z = tf.keras.layers.Dense(hidden, activation=tf.math.sin,
                                  kernel_initializer="glorot_uniform",
                                  name='dense_sine_1')(x_in)
        z = tf.keras.layers.Dense(hidden, activation=tf.math.sin,
                                  kernel_initializer="glorot_uniform", 
                                  name='dense_sine_2')(z)
    else:
        z = tf.keras.layers.Dense(hidden, activation="tanh",
                                  kernel_initializer="glorot_uniform",
                                  name='dense_tanh_1')(x_in)
        z = tf.keras.layers.Dense(hidden, activation="tanh",
                                  kernel_initializer="glorot_uniform",
                                  name='dense_tanh_2')(z)
    
    # Capa de salida
    network_output = tf.keras.layers.Dense(1, activation=None,
                                          kernel_initializer="glorot_uniform",
                                          name='network_output')(z)
    
    # Combinar todo: ψ(x) = [x*(1-x)] * [F_n(x)] * [N(x)]
    boundary_nodal = tf.keras.layers.Multiply(name='boundary_nodal_combination')(
        [boundary_enforcer, nodal_factor]
    )
    
    psi_output = tf.keras.layers.Multiply(name='final_output')(
        [boundary_nodal, network_output]
    )
    
    model = tf.keras.Model(inputs=x_in, outputs=psi_output, name=f'pinn_mode_{n}')
    return model

# ==================== CÁLCULO DE DERIVADAS ====================
def second_derivative(model, x):
    """
    Calcula la primera y segunda derivada usando GradientTape
    """
    x = tf.convert_to_tensor(x)
    x = tf.reshape(x, (-1, 1))
    
    with tf.GradientTape(persistent=True) as tape2:
        tape2.watch(x)
        with tf.GradientTape() as tape1:
            tape1.watch(x)
            psi = model(x)
        psi_x = tape1.gradient(psi, x)
    psi_xx = tape2.gradient(psi_x, x)
    
    del tape2
    return psi, psi_xx

# ==================== PÉRDIDAS ====================
def compute_losses(net, x_batch, E, lam):
    """
    Calcula las pérdidas totales, PDE y de normalización
    """
    psi, psi_xx = second_derivative(net, x_batch)
    
    # Residual de la ecuación de Schrödinger: ψ'' + Eψ = 0
    residual = psi_xx + E * psi
    LPDE = tf.reduce_mean(tf.square(residual))
    
    # Calcular la integral de normalización ∫|ψ|²dx
    psi_squared = tf.square(psi)
    x_squeezed = tf.squeeze(x_batch)
    psi_squared_squeezed = tf.squeeze(psi_squared)
    
    # Integración numérica usando la regla del trapecio
    dx = x_squeezed[1:] - x_squeezed[:-1]
    integrand_avg = 0.5 * (psi_squared_squeezed[1:] + psi_squared_squeezed[:-1])
    integral = tf.reduce_sum(integrand_avg * dx)
    
    # Pérdida de normalización: (∫|ψ|²dx - 1)²
    Lnorm = tf.square(integral - 1.0)
    
    # Pérdida total
    L = LPDE + lam * Lnorm
    
    return L, LPDE, Lnorm, integral

# ==================== ENTRENAMIENTO COMPLETO ====================
def run_one_mode_complete(n, save_dir=None, use_noise=True, use_kalman=True, force_retrain=False):
    """
    Versión completa con todas las características
    """
    if save_dir is None:
        save_dir = BASE_DIR
    
    # Crear subcarpeta para este modo y método
    method_name = "Ruido_Kalman" if (use_noise and use_kalman) else "Con_Ruido" if use_noise else "Baseline"
    mode_dir = os.path.join(save_dir, f"metodo_{method_name}", f"modo_{n}")
    os.makedirs(mode_dir, exist_ok=True)
    
    results_file = os.path.join(mode_dir, f"results_mode_{n}.pkl")
    figures_dir = os.path.join(mode_dir, "figuras")
    os.makedirs(figures_dir, exist_ok=True)
    
    # Verificar si ya existen resultados
    if not force_retrain and os.path.exists(results_file):
        print(f"📁 Cargando resultados existentes para n={n} ({method_name})")
        with open(results_file, 'rb') as f:
            return pickle.load(f)
    
    print(f"🚀 Entrenando n={n} ({method_name})...")
    
    # Energía y hiperparámetros
    E_exact = np.float32((n * math.pi)**2)
    USE_SINE = True if n >= 3 else False
    HIDDEN   = 128 if n >= 3 else 64
    N_col    = max(1024, 2048*n)
    EPOCHS   = 15000 if n >= 4 else (9000 if n==3 else (6000 if n==2 else 4000))
    LR0      = 3e-4  if n >= 4 else (5e-4 if n==3 else (7e-4 if n==2 else 1e-3))

    lam_hi, lam_lo = (300.0, 80.0) if n >= 3 else (40.0, 15.0 if n==2 else 10.0)
    
    # Red neuronal
    net = make_net(n=n, hidden=HIDDEN, use_sine=USE_SINE)
    
    # Puntos base para colación
    x_base = np.linspace(0, 1, N_col, dtype=np.float32)
    
    # Filtro de Kalman
    if use_kalman:
        lambda_kf = ParameterKalmanFilter(initial_value=lam_hi)
    
    # Optimizador
    lr_sched = tf.keras.optimizers.schedules.PolynomialDecay(
        initial_learning_rate=LR0, decay_steps=EPOCHS, end_learning_rate=LR0*0.1
    )
    opt = tf.keras.optimizers.Adam(learning_rate=lr_sched, clipnorm=1.0)

    # Historial
    loss_total, loss_pde, loss_norm, lambda_history = [], [], [], []

    # Entrenamiento
    for ep in range(1, EPOCHS+1):
        # ESTRATEGIA DE MUESTREO CON RUIDO
        if use_noise:
            noise_level = max(0.02 * (1 - ep/EPOCHS), 0.005)
            noise = np.random.normal(0, noise_level, N_col).astype(np.float32)
            x_col = np.clip(x_base + noise, 0, 1).reshape(-1, 1)
        else:
            x_col = x_base.reshape(-1, 1)
        
        x_batch = tf.constant(x_col)

        # ESTRATEGIA LAMBDA CON KALMAN
        if use_kalman and ep > EPOCHS//10:
            lam = lambda_kf.update(float(loss_pde[-1] if loss_pde else lam_hi))
            lam = max(lam, lam_lo)
        else:
            lam = lam_hi if ep < EPOCHS//3 else lam_lo
        
        lambda_history.append(lam)

        # Paso de entrenamiento
        with tf.GradientTape() as tape:
            L, LPDE, Lnorm, integral = compute_losses(net, x_batch, E_exact, lam)
        
        grads = tape.gradient(L, net.trainable_variables)
        opt.apply_gradients(zip(grads, net.trainable_variables))

        loss_total.append(float(L))
        loss_pde.append(float(LPDE))
        loss_norm.append(float(Lnorm))

        if ep % max(1000, EPOCHS//10) == 0:
            print(f"n={n} ({method_name}) | ep={ep:4d}/{EPOCHS} | LPDE={LPDE:.2e} | Lnorm={Lnorm:.2e} | ∫|ψ|²={integral:.4f}")

    # Evaluación final
    xs = np.linspace(0, 1, 2000, dtype=np.float32).reshape(-1, 1)
    psi_pred = net(xs).numpy().squeeze()
    psi_exact = (np.sqrt(2.0) * np.sin(n * math.pi * xs)).squeeze()

    # Alinear signo
    sign = np.sign(np.dot(psi_pred, psi_exact))
    psi_pred *= sign

    # Métricas
    l2_err = float(np.sqrt(np.mean((psi_pred - psi_exact)**2)))
    l1_err = float(np.mean(np.abs(psi_pred - psi_exact)))
    integ = float(integrate_simps(psi_pred**2, xs.squeeze()))
    
    # Resultados
    results = {
        'n': n,
        'xs': xs.squeeze(),
        'psi_pred': psi_pred,
        'psi_exact': psi_exact,
        'E_exact': float(E_exact),
        'L2_error': l2_err,
        'L1_error': l1_err,
        'integral': integ,
        'loss_total': loss_total,
        'loss_pde': loss_pde,
        'loss_norm': loss_norm,
        'lambda_history': lambda_history,
        'method': method_name,
        'training_params': {
            'use_noise': use_noise,
            'use_kalman': use_kalman,
            'hidden_units': HIDDEN,
            'epochs': EPOCHS,
            'learning_rate': LR0
        }
    }

    # Guardar
    with open(results_file, 'wb') as f:
        pickle.dump(results, f)
    
    print(f"✅ n={n} ({method_name}) completado - L2: {l2_err:.2e}")
    
    return results

# ==================== COMPARACIÓN ENTRE MÉTODOS ====================
def run_comparison_metodos(n_max=6):
    """Ejecuta la comparación entre los tres métodos para todos los modos"""
    print("🔄 INICIANDO COMPARACIÓN ENTRE MÉTODOS")
    print("=" * 60)
    
    methods = [
        {'name': 'Baseline', 'use_noise': False, 'use_kalman': False},
        {'name': 'Con_Ruido', 'use_noise': True, 'use_kalman': False},
        {'name': 'Ruido_Kalman', 'use_noise': True, 'use_kalman': True}
    ]
    
    all_results = {}
    
    for method in methods:
        print(f"\n🔍 Método: {method['name']}")
        print("-" * 40)
        
        method_results = []
        for n in range(1, n_max + 1):
            try:
                results = run_one_mode_complete(
                    n=n, 
                    use_noise=method['use_noise'], 
                    use_kalman=method['use_kalman'],
                    force_retrain=False
                )
                method_results.append(results)
                print(f"   ✅ n={n} completado")
            except Exception as e:
                print(f"   ❌ n={n} falló: {e}")
                # Continuar con el siguiente modo
                continue
        
        all_results[method['name']] = method_results
    
    # Generar reporte comparativo
    print("\n" + "=" * 80)
    print("REPORTE COMPARATIVO - TODOS LOS MÉTODOS")
    print("=" * 80)
    print(f"{'Método':<15} {'n':<3} {'L2 Error':<12} {'L1 Error':<12} {'∫|ψ|²':<8} {'Épocas':<8}")
    print("-" * 80)
    
    for method_name, results_list in all_results.items():
        for results in results_list:
            n = results['n']
            l2 = results['L2_error']
            l1 = results['L1_error']
            integral = results['integral']
            epochs = len(results['loss_total'])
            
            print(f"{method_name:<15} {n:<3} {l2:<12.2e} {l1:<12.2e} {integral:<8.4f} {epochs:<8}")
    
    return all_results

# ==================== GRÁFICAS COMPARATIVAS ====================
def generar_graficas_comparativas(all_results, n_max=6):
    """Genera todas las gráficas comparativas"""
    print("\n📊 GENERANDO GRÁFICAS COMPARATIVAS")
    
    comparison_dir = os.path.join(BASE_DIR, "comparacion_metodos")
    os.makedirs(comparison_dir, exist_ok=True)
    
    # 1. Gráfica de errores L2 por método
    plt.figure(figsize=(12, 8))
    
    for method_name, results_list in all_results.items():
        n_values = [r['n'] for r in results_list]
        l2_errors = [r['L2_error'] for r in results_list]
        
        plt.semilogy(n_values, l2_errors, 'o-', label=method_name, linewidth=3, markersize=8)
    
    plt.xlabel('Número cuántico n', fontsize=12)
    plt.ylabel('Error L2', fontsize=12)
    plt.title('Comparación de Métodos: Error L2 vs Modo (n=1-6)', fontsize=14)
    plt.legend(fontsize=11)
    plt.grid(True, alpha=0.3)
    plt.xticks(range(1, n_max + 1))
    plt.savefig(os.path.join(comparison_dir, 'comparacion_errores_l2_todos.png'), dpi=300, bbox_inches='tight')
    plt.close()
    
    # 2. Gráfica de curvas de aprendizaje para cada modo
    for n in range(1, n_max + 1):
        plt.figure(figsize=(14, 8))
        
        for method_name, results_list in all_results.items():
            for results in results_list:
                if results['n'] == n:
                    epochs = range(1, len(results['loss_total']) + 1)
                    plt.semilogy(epochs, results['loss_total'], label=f"{method_name}", linewidth=2.5)
        
        plt.xlabel('Época', fontsize=12)
        plt.ylabel('Pérdida Total', fontsize=12)
        plt.title(f'Comparación de Curvas de Aprendizaje - Modo n={n}', fontsize=14)
        plt.legend(fontsize=11)
        plt.grid(True, alpha=0.3)
        plt.savefig(os.path.join(comparison_dir, f'comparacion_aprendizaje_n{n}.png'), dpi=300, bbox_inches='tight')
        plt.close()
    
    # 3. Gráfica de funciones de onda para cada método (n=1-6)
    for method_name, results_list in all_results.items():
        plt.figure(figsize=(15, 10))
        
        for results in results_list:
            n = results['n']
            xs = results['xs']
            psi_pred = results['psi_pred']
            psi_exact = results['psi_exact']
            
            # Desplazar verticalmente para visualización
            offset = 2.5 * (n - 1)
            plt.plot(xs, psi_pred + offset, label=f'PINN ψ_{n}', linewidth=2)
            plt.plot(xs, psi_exact + offset, '--', alpha=0.7, linewidth=1.5, color='gray')
        
        plt.xlabel('x', fontsize=12)
        plt.ylabel('ψ(x) (desplazadas)', fontsize=12)
        plt.title(f'Funciones de Onda - Método {method_name} (n=1-6)', fontsize=14)
        plt.legend(fontsize=10)
        plt.grid(True, alpha=0.3)
        plt.savefig(os.path.join(comparison_dir, f'funciones_onda_{method_name}.png'), dpi=300, bbox_inches='tight')
        plt.close()
    
    # 4. Gráfica de normalización por método
    plt.figure(figsize=(12, 8))
    
    for method_name, results_list in all_results.items():
        n_values = [r['n'] for r in results_list]
        integrals = [r['integral'] for r in results_list]
        
        plt.plot(n_values, integrals, 's-', label=method_name, linewidth=2, markersize=6)
    
    plt.axhline(y=1.0, color='red', linestyle='--', alpha=0.7, label='Valor objetivo (1.0)')
    plt.xlabel('Número cuántico n', fontsize=12)
    plt.ylabel('∫|ψ|²dx', fontsize=12)
    plt.title('Normalización de las Funciones de Onda por Método', fontsize=14)
    plt.legend(fontsize=11)
    plt.grid(True, alpha=0.3)
    plt.xticks(range(1, n_max + 1))
    plt.savefig(os.path.join(comparison_dir, 'comparacion_normalizacion.png'), dpi=300, bbox_inches='tight')
    plt.close()
    
    print(f"✅ Gráficas comparativas guardadas en: {comparison_dir}")

# ==================== FUNCIONES DE UTILIDAD ====================
def cargar_resultados_metodo(n, method_name, base_dir=BASE_DIR):
    """Carga resultados específicos de un método"""
    load_dir = os.path.join(base_dir, f"metodo_{method_name}", f"modo_{n}")
    results_file = os.path.join(load_dir, f"results_mode_{n}.pkl")
    
    if os.path.exists(results_file):
        with open(results_file, 'rb') as f:
            return pickle.load(f)
    else:
        print(f"❌ No se encontraron resultados para n={n}, método {method_name}")
        return None

def listar_resultados_disponibles(base_dir=BASE_DIR):
    """Lista todos los resultados disponibles"""
    print(f"\n📁 RESULTADOS DISPONIBLES en: {base_dir}")
    print("=" * 50)
    
    methods = ['Baseline', 'Con_Ruido', 'Ruido_Kalman']
    
    for method in methods:
        method_dir = os.path.join(base_dir, f"metodo_{method}")
        if os.path.exists(method_dir):
            print(f"\n🔹 Método: {method}")
            for n in range(1, 7):
                results_file = os.path.join(method_dir, f"modo_{n}", f"results_mode_{n}.pkl")
                if os.path.exists(results_file):
                    with open(results_file, 'rb') as f:
                        results = pickle.load(f)
                    print(f"   n={n} | L2: {results['L2_error']:.2e} | ∫|ψ|²: {results['integral']:.4f}")

# ==================== EJECUCIÓN PRINCIPAL ====================
if __name__ == "__main__":
    print("🎯 PINN AVANZADO - COMPARACIÓN COMPLETA DE MÉTODOS")
    print(f"📁 Directorio base: {BASE_DIR}")
    print("=" * 70)
    
    # Ejecutar comparación completa entre métodos para n=1-6
    all_results = run_comparison_metodos(n_max=6)
    
    # Generar gráficas comparativas
    generar_graficas_comparativas(all_results, n_max=6)
    
    # Mostrar resultados disponibles
    listar_resultados_disponibles()
    
    print(f"\n🎉 ¡COMPARACIÓN COMPLETADA!")
    print(f"📊 Resultados guardados en: {BASE_DIR}")
    print("📈 Gráficas comparativas en: comparacion_metodos/")
    print("🔍 Estructura generada:")
    print("   - metodo_Baseline/modo_1...6/")
    print("   - metodo_Con_Ruido/modo_1...6/") 
    print("   - metodo_Ruido_Kalman/modo_1...6/")
    print("   - comparacion_metodos/ (gráficas comparativas)")

📁 Directorio base: /home/david/Schr-dingerPINNsUQValidation/PrimeraFase/ValidacionPozoInfinito/FiltroKalman/ResultadosK
✅ Estructura de carpetas creada
🎯 PINN AVANZADO - COMPARACIÓN COMPLETA DE MÉTODOS
📁 Directorio base: /home/david/Schr-dingerPINNsUQValidation/PrimeraFase/ValidacionPozoInfinito/FiltroKalman/ResultadosK
🔄 INICIANDO COMPARACIÓN ENTRE MÉTODOS

🔍 Método: Baseline
----------------------------------------
🚀 Entrenando n=1 (Baseline)...
n=1 (Baseline) | ep=1000/4000 | LPDE=1.76e-03 | Lnorm=4.76e-09 | ∫|ψ|²=0.9999
n=1 (Baseline) | ep=2000/4000 | LPDE=9.63e-04 | Lnorm=1.33e-08 | ∫|ψ|²=0.9999
n=1 (Baseline) | ep=3000/4000 | LPDE=4.04e-04 | Lnorm=2.45e-09 | ∫|ψ|²=1.0000
n=1 (Baseline) | ep=4000/4000 | LPDE=1.73e-04 | Lnorm=5.68e-10 | ∫|ψ|²=1.0000
✅ n=1 (Baseline) completado - L2: 4.51e-05
   ✅ n=1 completado
🚀 Entrenando n=2 (Baseline)...


/tmp/ipykernel_13838/2046432043.py:32: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  return np.trapz(y, x)


n=2 (Baseline) | ep=1000/6000 | LPDE=5.83e-04 | Lnorm=3.04e-08 | ∫|ψ|²=0.9998
n=2 (Baseline) | ep=2000/6000 | LPDE=2.15e-04 | Lnorm=1.11e-08 | ∫|ψ|²=0.9999
n=2 (Baseline) | ep=3000/6000 | LPDE=1.49e-04 | Lnorm=1.51e-08 | ∫|ψ|²=1.0001
n=2 (Baseline) | ep=4000/6000 | LPDE=5.49e-05 | Lnorm=6.39e-10 | ∫|ψ|²=1.0000
n=2 (Baseline) | ep=5000/6000 | LPDE=2.34e-05 | Lnorm=3.07e-10 | ∫|ψ|²=1.0000
n=2 (Baseline) | ep=6000/6000 | LPDE=1.17e-05 | Lnorm=1.11e-11 | ∫|ψ|²=1.0000
✅ n=2 (Baseline) completado - L2: 1.82e-05
   ✅ n=2 completado
🚀 Entrenando n=3 (Baseline)...
n=3 (Baseline) | ep=1000/9000 | LPDE=6.70e-03 | Lnorm=1.27e-05 | ∫|ψ|²=0.9964
n=3 (Baseline) | ep=2000/9000 | LPDE=2.88e-03 | Lnorm=1.10e-05 | ∫|ψ|²=0.9967
n=3 (Baseline) | ep=3000/9000 | LPDE=1.56e-03 | Lnorm=7.68e-06 | ∫|ψ|²=0.9972
n=3 (Baseline) | ep=4000/9000 | LPDE=3.48e-03 | Lnorm=8.07e-06 | ∫|ψ|²=1.0028
n=3 (Baseline) | ep=5000/9000 | LPDE=4.77e-03 | Lnorm=1.30e-05 | ∫|ψ|²=0.9964
n=3 (Baseline) | ep=6000/9000 | LPDE=9.54e-04 | 

In [10]:
import os, pickle, csv
from collections import defaultdict

# >>> Ajusta si cambiaste la ruta base <<<
BASE_DIR = "/home/david/Schr-dingerPINNsUQValidation/PrimeraFase/ValidacionPozoInfinito/FiltroKalman/ResultadosK"
METHODS = ["Baseline", "Con_Ruido", "Ruido_Kalman"]
N_MAX = 6

COMP_DIR = os.path.join(BASE_DIR, "comparacion_metodos")
os.makedirs(COMP_DIR, exist_ok=True)

def load_result(method: str, n: int):
    p = os.path.join(BASE_DIR, f"metodo_{method}", f"modo_{n}", f"results_mode_{n}.pkl")
    if not os.path.isfile(p):
        return None
    with open(p, "rb") as f:
        return pickle.load(f)

def fmt_sci(x):  return "-" if x is None else f"{x:.2e}"
def fmt_flt(x):  return "-" if x is None else f"{x:.6f}"

# === Cargar todo ===
rows = []
for m in METHODS:
    for n in range(1, N_MAX+1):
        d = load_result(m, n)
        if d is None: 
            continue
        rows.append({
            "method": d.get("method", m),
            "n": int(d["n"]),
            "L2": float(d["L2_error"]),
            "L1": float(d["L1_error"]),
            "I":  float(d["integral"]),
            "NormErr": abs(float(d["integral"])-1.0),
        })

if not rows:
    raise SystemExit("No encontré .pkl. ¿Corriste los experimentos?")

# === Imprimir por método (tabla bonita) ===
print("\n" + "="*88)
print("RESUMEN POR MÉTODO (n, L2, L1, ∫|ψ|², |∫|ψ|²-1|)")
print("="*88)
rows_sorted = sorted(rows, key=lambda r: (r["method"], r["n"]))
current = None
for r in rows_sorted:
    if r["method"] != current:
        current = r["method"]
        print(f"\n[{current}]")
        print(f"{'n':>2}  {'L2':>12}  {'L1':>12}  {'Integral':>10}  {'|∫-1|':>10}")
    print(f"{r['n']:>2}  {fmt_sci(r['L2']):>12}  {fmt_sci(r['L1']):>12}  {fmt_flt(r['I']):>10}  {fmt_sci(r['NormErr']):>10}")

# === Pivot L2 por n (filas) y método (columnas) ===
by_n = defaultdict(dict)
for r in rows:
    by_n[r["n"]][r["method"]] = r

print("\n" + "="*88)
print("L2 POR MODO (filas) Y MÉTODO (columnas)")
print("="*88)
header = f"{'n':>2}  {'Baseline':>12}  {'Con_Ruido':>12}  {'Ruido_Kalman':>12}"
print(header)
print("-"*len(header))
for n in range(1, N_MAX+1):
    a = by_n.get(n, {}).get("Baseline")
    b = by_n.get(n, {}).get("Con_Ruido")
    c = by_n.get(n, {}).get("Ruido_Kalman")
    print(f"{n:>2}  {fmt_sci(a['L2']) if a else '-':>12}  {fmt_sci(b['L2']) if b else '-':>12}  {fmt_sci(c['L2']) if c else '-':>12}")

# === Guardar CSV plano ===
csv_path = os.path.join(COMP_DIR, "resumen_resultados.csv")
with open(csv_path, "w", newline="") as f:
    w = csv.writer(f)
    w.writerow(["method","n","L2","L1","integral","norm_error"])
    for r in rows_sorted:
        w.writerow([r["method"], r["n"], f"{r['L2']:.6e}", f"{r['L1']:.6e}", f"{r['I']:.6f}", f"{r['NormErr']:.6e}"])
print(f"\n[OK] CSV -> {csv_path}")

# === (Opcional) .tex con dos tablas (L2 y normalización) ===
tex_path = os.path.join(COMP_DIR, "tabla_L2_norm.tex")
with open(tex_path, "w") as f:
    f.write("% Auto-generado: incluir con \\input{...}\n\n")
    # L2
    f.write("\\begin{table}[H]\\centering\\small\n")
    f.write("\\begin{tabular}{@{}lccc@{}}\\toprule\n")
    f.write("\\textbf{Modo $n$} & \\textbf{Baseline $L_2$} & \\textbf{Con\\_Ruido $L_2$} & \\textbf{Ruido\\_Kalman $L_2$}\\\\\\midrule\n")
    for n in range(1, N_MAX+1):
        a = by_n.get(n, {}).get("Baseline"); b = by_n.get(n, {}).get("Con_Ruido"); c = by_n.get(n, {}).get("Ruido_Kalman")
        f.write(f"{n} & {fmt_sci(a['L2']) if a else '-'} & {fmt_sci(b['L2']) if b else '-'} & {fmt_sci(c['L2']) if c else '-'}\\\\\n")
    f.write("\\bottomrule\\end{tabular}\n\\caption{Error $L_2$ por modo y método.}\\label{tab:l2_methods}\\end{table}\n\n")
    # Normalización (|∫-1|)
    f.write("\\begin{table}[H]\\centering\\small\n")
    f.write("\\begin{tabular}{@{}lccc@{}}\\toprule\n")
    f.write("\\textbf{Modo $n$} & \\textbf{Baseline $|\\int|\\psi|^2-1|$} & \\textbf{Con\\_Ruido $|\\int|\\psi|^2-1|$} & \\textbf{Ruido\\_Kalman $|\\int|\\psi|^2-1|$}\\\\\\midrule\n")
    for n in range(1, N_MAX+1):
        a = by_n.get(n, {}).get("Baseline"); b = by_n.get(n, {}).get("Con_Ruido"); c = by_n.get(n, {}).get("Ruido_Kalman")
        f.write(f"{n} & {fmt_sci(a['NormErr']) if a else '-'} & {fmt_sci(b['NormErr']) if b else '-'} & {fmt_sci(c['NormErr']) if c else '-'}\\\\\n")
    f.write("\\bottomrule\\end{tabular}\n\\caption{Error de normalización $|\\int_0^1|\\psi|^2dx-1|$ por modo y método.}\\label{tab:norm_methods}\\end{table}\n")
print(f"[OK] LaTeX -> {tex_path}\n")
print("Inserta en tu .tex con:")
print(f"\\input{{{tex_path}}}")



RESUMEN POR MÉTODO (n, L2, L1, ∫|ψ|², |∫|ψ|²-1|)

[Baseline]
 n            L2            L1    Integral       |∫-1|
 1      4.51e-05      3.83e-05    0.999991    8.76e-06
 2      1.82e-05      1.39e-05    0.999990    1.01e-05
 3      4.62e-05      3.69e-05    1.000002    1.67e-06
 4      5.68e-05      4.27e-05    1.000004    4.41e-06
 5      4.85e-05      3.54e-05    0.999992    7.87e-06
 6      4.15e-05      3.20e-05    0.999993    6.68e-06

[Con_Ruido]
 n            L2            L1    Integral       |∫-1|
 1      6.68e-05      5.36e-05    1.000059    5.88e-05
 2      9.70e-05      8.63e-05    0.999813    1.87e-04
 3      1.65e-04      1.25e-04    0.999797    2.03e-04
 4      4.30e-04      3.71e-04    0.999175    8.25e-04
 5      1.47e-03      1.32e-03    0.997072    2.93e-03
 6      3.31e-04      2.91e-04    1.000646    6.46e-04

[Ruido_Kalman]
 n            L2            L1    Integral       |∫-1|
 1      3.80e-05      3.12e-05    1.000013    1.31e-05
 2      8.99e-05      8.11e-0